# Word2Vec: just for fun

[Trello home](https://trello.com/c/lDKOCrOr/11-w2v-cosine-similarity)

Сурсы по word2vec:
* [Слайды](http://www.machinelearning.ru/wiki/images/9/9d/BartunovLSM.pdf), которые я не смотрел
* [Habr. Word2vec с картинками](https://habr.com/ru/post/446530/)
* [Что такое w2v. Статья](https://neurohive.io/ru/osnovy-data-science/word2vec-vektornye-predstavlenija-slov-dlja-mashinnogo-obuchenija/)
* [Презентация крутого лектора](https://docs.google.com/presentation/d/1JNuhCuWyInu6aUrAK4MmLd5TZATDptPUsUx1z2JcYT0/mobilepresent#slide=id.p)
* [Jupyter tutorial **word2vec**](https://nbviewer.jupyter.org/github/danielfrg/word2vec/blob/master/examples/word2vec.ipynb) - у меня не заработала эта библиотека
* [Мужик играет с **word2vec** на русском](https://alexeytrudov.com/web-marketing/seo/poigralsya-s-word2vec-mashinnoe-obuchenie.html) - у меня так не получилось
* [Словарик с векторами русских слов](https://nlpub.mipt.ru/Russian_Distributional_Thesaurus)

### Подготовка данных
Для начала развлекусь и сделаю **word2vec** для этого странного набора твитов, а потом уже возьму сторонние вектора и буду использовать их

In [22]:
import pandas as pd
import numpy as np
import re

In [24]:
df_pos = pd.read_csv('../../data/twitts/positive.csv', sep=';', header=None)
df_neg = pd.read_csv('../../data/twitts/negative.csv', sep=';', header=None)

In [25]:
df = df_pos.append(df_neg, ignore_index=True)[[3]].rename({3:'tweet'}, axis=1)

In [26]:
df.head()

,tweet
0,"@first_timee хоть я и школота, но поверь, у на..."
1,"Да, все-таки он немного похож на него. Но мой ..."
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...
3,"RT @digger2912: ""Кто то в углу сидит и погибае..."
4,@irina_dyshkant Вот что значит страшилка :D\nН...


In [27]:
for v in df.tweet[:10].values:
    print(v)

@first_timee хоть я и школота, но поверь, у нас то же самое :D общество профилирующий предмет типа)
Да, все-таки он немного похож на него. Но мой мальчик все равно лучше:D
RT @KatiaCheh: Ну ты идиотка) я испугалась за тебя!!!
RT @digger2912: "Кто то в углу сидит и погибает от голода, а мы ещё 2 порции взяли, хотя уже и так жрать не хотим" :DD http://t.co/GqG6iuE2…
@irina_dyshkant Вот что значит страшилка :D
Но блин,посмотрев все части,у тебя создастся ощущение,что авторы курили что-то :D
ну любишь или нет? — Я не знаю кто ты бля:D http://t.co/brf9eNg1U6
RT @SpoonLamer: Ох,900 :D ну это конечно же @twinkleAYO . Чтобы у нее было много друзей, ведь она такая мимими &lt;3
RT @veregijytaqo: У тебя есть ухажёр? Нет - мои уши не кто не жрёт :D
Поприветствуем моего нового читателя @Alexey1789 ;)
Теперь у меня есть частичка Сиднея :) #Sydney #SydneyOperaHouse http://t.co/agnto3CZEi


Почистить

In [28]:
df.tweet = df['tweet'].apply(lambda x: re.sub('[\Wa-zA-Z_\d]+', ' ', x) )

In [29]:
df.tweet = df['tweet'].apply(lambda x: x.lower().strip() )

In [30]:
for v in df.tweet[:10].values:
    print(v)

хоть я и школота но поверь у нас то же самое общество профилирующий предмет типа
да все таки он немного похож на него но мой мальчик все равно лучше
ну ты идиотка я испугалась за тебя
кто то в углу сидит и погибает от голода а мы ещё порции взяли хотя уже и так жрать не хотим
вот что значит страшилка но блин посмотрев все части у тебя создастся ощущение что авторы курили что то
ну любишь или нет я не знаю кто ты бля
ох ну это конечно же чтобы у нее было много друзей ведь она такая мимими
у тебя есть ухажёр нет мои уши не кто не жрёт
поприветствуем моего нового читателя
теперь у меня есть частичка сиднея


In [102]:
df = df.drop_duplicates()

In [103]:
df.shape

(210983, 1)

In [115]:
df.to_csv("clean_strings", index=False, header=None)

### Word2Vec part

Нашёл библиотеку [GenSim](https://radimrehurek.com/gensim/auto_examples/index.html)

In [31]:
from gensim.test.utils import datapath
from gensim import utils
import gensim.models

In [14]:
class MyCorpus(object):
    """An interator that yields sentences (lists of str)."""

    def __iter__(self):
        corpus_path = datapath('/home/nikitap/projects/python/augmentations_comptech/Word2Vec/clean_strings')#full path
        for line in open(corpus_path):
            # assume there's one document per line, tokens separated by whitespace
            yield utils.simple_preprocess(line)

In [17]:
sentences = MyCorpus()
model = gensim.models.Word2Vec(sentences=sentences)

в тот момент, когда я хотел всё закрыть, модель дообучилась. Кажется, это знак

In [23]:
model

In [24]:
model.save('/home/nikitap/projects/python/augmentations_comptech/Word2Vec/model')

Сохранил модель на будущее, чтоб не тратить ещё кучу времени на обучение

In [32]:
new_model = gensim.models.Word2Vec.load('/home/nikitap/projects/python/augmentations_comptech/Word2Vec/model')

In [4]:
new_model

In [5]:
for i, word in enumerate(new_model.wv.vocab):
    if i == 10:
        break
    print(word)

хоть
школота
но
поверь
нас
то
же
самое
общество
предмет


In [6]:
print(new_model.wv.most_similar(positive=['путин',"навальный"], topn=5))

[('медведев', 0.9664825797080994), ('миронов', 0.9582897424697876), ('президент', 0.9568126201629639), ('кличко', 0.9522696733474731), ('ленину', 0.9514321088790894)]


In [7]:
new_model.wv.similar_by_word('путин')

[('президент', 0.9185464382171631),
 ('медведев', 0.8956725597381592),
 ('гуф', 0.8938204050064087),
 ('бенедикт', 0.8936036825180054),
 ('мамонтов', 0.8918490409851074),
 ('калашников', 0.8917428255081177),
 ('вспоминай', 0.8897215723991394),
 ('жуйку', 0.8889330625534058),
 ('существовании', 0.886281430721283),
 ('врет', 0.8853999972343445)]

In [15]:
new_model.similar_by_vector(new_model['путин'] - new_model["мужчина"] + new_model["женщина"] )

/home/nikitap/programs/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.
/home/nikitap/programs/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similar_by_vector` (Method will be removed in 4.0.0, use self.wv.similar_by_vector() instead).
  """Entry point for launching an IPython kernel.


[('знакомая', 0.7293884754180908),
 ('страна', 0.7132641077041626),
 ('ваша', 0.7066949605941772),
 ('беда', 0.7000047564506531),
 ('большая', 0.6980278491973877),
 ('фамилия', 0.6976913213729858),
 ('умная', 0.6929706335067749),
 ('главная', 0.6923493146896362),
 ('огромная', 0.6899139881134033),
 ('верная', 0.68845534324646)]

гуф? O_O
___________

### Теперь беру уже обученную модель с [сайта](https://nlpub.mipt.ru/Russian_Distributional_Thesaurus)

In [3]:
mipt_model = gensim.models.KeyedVectors.load_word2vec_format(
    '/home/nikitap/projects/python/augmentations_comptech/Word2Vec/mipt_vecs.w2v', binary=True, unicode_errors='ignore')

Подгружается довольно долго

In [4]:
mipt_model.similar_by_word('путин')

[('путина', 0.9378689527511597),
 ('ельцин', 0.9320881366729736),
 ('ельцина', 0.906755805015564),
 ('чубайс', 0.9058520793914795),
 ('черномырдин', 0.9044474363327026),
 ('путиным', 0.8942527770996094),
 ('путину', 0.8915537595748901),
 ('лужков', 0.8785982131958008),
 ('горбачев', 0.8739296197891235),
 ('степашин', 0.8706163763999939)]

In [34]:
mipt_model.similar_by_vector( mipt_model["россия"] - mipt_model["нефть"] )

[('сочувствий', 0.4839920699596405),
 ('коялович', 0.4141654968261719),
 ('нападков', 0.4141131639480591),
 ('кохановская', 0.4140699505805969),
 ('карамзинских', 0.4127064347267151),
 ('сознала', 0.4126369059085846),
 ('верною', 0.40655913949012756),
 ('православною', 0.403412401676178),
 ('карамзинской', 0.3995151221752167),
 ('трубецкой', 0.39942246675491333)]

можно попробовать дообучить эту модель на твитах, но это как-нибудь в другой раз